In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Set up model parameters

# model_name = "meta-llama/Llama-3.2-3B-Instruct"
# max_tokens = 2048     # Maximum tokens to generate in response
# temperature = 0.6    # Sampling temperature
# top_p = 0.9
# top_k = 50
# repetition_penalty = 1.0

model_name = "Qwen/Qwen2.5-7B-Instruct"  
# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
max_tokens = 2048     # Maximum tokens to generate in response
temperature = 1.0    # Sampling temperature



# Load the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:

#### system prompts

system_prompt = """\
You are a helpful assistant who plays chess professionally.
The assistant first thinks through the reasoning process internally and then provides the user with the best move.
The reasoning process and the answer must be enclosed within <think> </think> and <answer> </answer> tags, respectively.
The reasoning process should describe how you analyze the position and decide on the best move.
The answer must be in SAN notation, strictly using the moving piece and the destination square (e.g., Nf3, Rxf2, c5). 
Now, the user provides a FEN string and a list of legal moves for the given board.
After analyzing the position, clearly state the best move in SAN notation within <answer> </answer> tags. i.e., <answer> Nf3 </answer>
"""


qa_system_prompt = """\
You are a helpful assistant who plays chess professionally.
"""

#### user prompts

user_prompt = """\
Previous moves: 1. e4 c6 2. d4 d5 3. Nc3 dxe4 4. Nxe4 Nf6 5. Nxf6+ exf6 6. Nf3 Bd6 7. Bd3 O-O 8. O-O Bg4 9. c3 Nd7 10. h3 Bh5 11. Re1 Qc7 12. Be2 Rfe8 13. Nd2 Bxe2 14. Rxe2 f5 15. Nc4 Nf6 16. Bg5 Bh2+ 17. Kh1 Ne4 18. Bd2 Nxd2 19. Qxd2 Bf4 20. Rae1 Kf8 21. Qd3 g6 22. g3 Bg5 23. h4 Bf6 24. Kg2 Rxe2 25. Rxe2 Rd8 26. Qe3 h5 27. Ne5 Kg7 28. Nd3 b6 29. Qf4 Qd7 30. Ne5 Qd5+ 31. Qf3 Bxe5 32. Qxd5 Rxd5 33. Rxe5 Rxe5 34. dxe5 f6 35. f4 fxe5\nCurrent FEN string: 8/p5k1/1pp3p1/4pp1p/5P1P/2P3P1/PP4K1/8 w - - 0 36\nLegal moves: Kh3 Kf3 Kh2 Kf2 Kh1 Kg1 Kf1 fxe5 g4 c4 b3 a3 b4 a4\n
"""

qa_user_prompt = """\
This is something related to chess.
What do you think this is?

a8: r b8: - c8: - d8: - e8: - f8: - g8: - h8: k a7: p b7: p c7: - d7: - e7: r f7: - g7: - h7: p a6: - b6: - c6: - d6: - e6: R f6: p g6: - h6: Q a5: - b5: - c5: - d5: p e5: - f5: - g5: - h5: - a4: - b4: - c4: - d4: - e4: - f4: - g4: - h4: - a3: - b3: N c3: - d3: P e3: - f3: - g3: b h3: - a2: P b2: q c2: P d2: - e2: - f2: - g2: P h2: P a1: - b1: - c1: - d1: - e1: - f1: - g1: - h1: K
"""

assistant_prompt = """\
Let's think step by step. <think>
"""



# Define a sample conversation (list of messages)
conversation = [
    {"role": "system", "content": qa_system_prompt},
    {"role": "user", "content": qa_user_prompt},
]

# Tokenize the prompt
inputs = tokenizer.apply_chat_template(
            conversation, 
            tokenize=True, 
            add_generation_prompt=True,
            return_tensors="pt").to(device)

# concat assistant related prompts
# inputs = torch.cat([inputs, tokenizer(assistant_prompt, return_tensors="pt").to(device)['input_ids']], dim=-1)
print(tokenizer.decode(inputs[0]))


<|im_start|>system
You are a helpful assistant who plays chess professionally.
<|im_end|>
<|im_start|>user
This is something related to chess.
What do you think this is?

a8: r b8: - c8: - d8: - e8: - f8: - g8: - h8: k a7: p b7: p c7: - d7: - e7: r f7: - g7: - h7: p a6: - b6: - c6: - d6: - e6: R f6: p g6: - h6: Q a5: - b5: - c5: - d5: p e5: - f5: - g5: - h5: - a4: - b4: - c4: - d4: - e4: - f4: - g4: - h4: - a3: - b3: N c3: - d3: P e3: - f3: - g3: b h3: - a2: P b2: q c2: P d2: - e2: - f2: - g2: P h2: P a1: - b1: - c1: - d1: - e1: - f1: - g1: - h1: K
<|im_end|>
<|im_start|>assistant



In [5]:
# Generate the response
outputs = model.generate(
    inputs,
    max_new_tokens=max_tokens,
    temperature=temperature,
    do_sample=True,
)

# Decode the generated tokens to text
full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(full_output)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system
You are a helpful assistant who plays chess professionally.

user
This is something related to chess.
What do you think this is?

a8: r b8: - c8: - d8: - e8: - f8: - g8: - h8: k a7: p b7: p c7: - d7: - e7: r f7: - g7: - h7: p a6: - b6: - c6: - d6: - e6: R f6: p g6: - h6: Q a5: - b5: - c5: - d5: p e5: - f5: - g5: - h5: - a4: - b4: - c4: - d4: - e4: - f4: - g4: - h4: - a3: - b3: N c3: - d3: P e3: - f3: - g3: b h3: - a2: P b2: q c2: P d2: - e2: - f2: - g2: P h2: P a1: - b1: - c1: - d1: - e1: - f1: - g1: - h1: K

assistant
The board you've described appears to be a starting position for a chess game with some pieces missing or mispositioned. Let's analyze the setup step-by-step:

```
 a8 r  b8 -  c8 -  d8 -  e8 -  f8 -  g8 -  h8 k
 a7 p  b7 -  c7 -  d7 -  e7 r  f7 -  g7 -  h7 p
 a6 -  b6 -  c6 -  d6 -  e6 R  f6 p  g6 -  h6 Q
 a5 -  b5 -  c5 -  d5 p  e5 -  f5 -  g5 -  h5 -
 a4 -  b4 -  c4 -  d4 -  e4 -  f4 -  g4 -  h4 -
 a3 -  b3 N  c3 -  d3 P  e3 -  f3 -  g3 b  h3 -
 a2 p  b2 q  c2 

In [2]:
print("a8: r b8: - c8: - d8: - e8: - f8: - g8: - h8: k\na7: p b7: p c7: - d7: - e7: r f7: - g7: - h7: p\na6: - b6: - c6: - d6: - e6: R f6: p g6: - h6: Q\na5: - b5: - c5: - d5: p e5: - f5: - g5: - h5: -\na4: - b4: - c4: - d4: - e4: - f4: - g4: - h4: -\na3: - b3: N c3: - d3: P e3: - f3: - g3: b h3: -\na2: P b2: q c2: P d2: - e2: - f2: - g2: P h2: P\na1: - b1: - c1: - d1: - e1: - f1: - g1: - h1: K")

a8: r b8: - c8: - d8: - e8: - f8: - g8: - h8: k
a7: p b7: p c7: - d7: - e7: r f7: - g7: - h7: p
a6: - b6: - c6: - d6: - e6: R f6: p g6: - h6: Q
a5: - b5: - c5: - d5: p e5: - f5: - g5: - h5: -
a4: - b4: - c4: - d4: - e4: - f4: - g4: - h4: -
a3: - b3: N c3: - d3: P e3: - f3: - g3: b h3: -
a2: P b2: q c2: P d2: - e2: - f2: - g2: P h2: P
a1: - b1: - c1: - d1: - e1: - f1: - g1: - h1: K
